In [ ]:
import pandas as pd
import numpy as np
from SFFirePredLibraries import preprocessInput


Reading training file

In [ ]:
df = pd.read_csv('Fire_Incidents.tsv', sep='\t')
df.describe()


Preprocessing train data frame

In [ ]:
df_feature = preprocessInput(df)
filter_df = df_feature[df_feature['Suppression Personnel'] <= 100]
filter_df.shape
filter_df['Suppression Personnel'].value_counts()
df_Y = filter_df['Suppression Personnel']
filter_df = filter_df.drop(['Suppression Personnel'],axis=1)
df_Y.shape


Creating CSR matrix

In [ ]:
from scipy.sparse import csr_matrix

def dataframetoCSRmatrix(df):
    nrows = len(df)
    nc = len(df.columns)
    idx = {}
    tid = 0
    nnz = nc * nrows
    
    cols= df.columns
    
    for col in cols:
        df[col] = df[col].apply(str)
        for name in df[col].unique():
            idx[col+name] = tid
            tid += 1
    
    ncols = len(idx)
    
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.int)
    ptr = np.zeros(nrows+1, dtype=np.int)
    
    i=0
    n=0
    
    for index,row in df.iterrows():
        for j,col in enumerate(cols):
            ind[j+n] = idx[col+row[col]]
            val[j+n] = 1
        ptr[i+1] = ptr[i] + nc
        n += nc
        i += 1
    
    mat = csr_matrix((val,ind,ptr), shape=(nrows,ncols), dtype=np.int)
    mat.sort_indices()   
    
    return mat

In [ ]:
mat1 = dataframetoCSRmatrix(filter_df)

Apply Ridge Regression Algorithm

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_squared_log_error

In [ ]:
df_feature_train, df_feature_test, df_y_train, df_y_test = train_test_split(mat1, df_Y, test_size = 0.2, random_state = 0)


# rfreg = RandomForestRegressor(n_estimators= 10,max_depth=1, random_state=42)
# rfreg.fit(df_feature_train, df_y_train)


# predicted_y_rf = rfreg.predict(df_feature_test)


# set(predicted_y_rf)



In [ ]:
from sklearn.linear_model import Ridge
reg_Ridge = Ridge(alpha = 0.000001)
reg_Ridge.fit(df_feature_train, df_y_train)

y_pred_list_Ridge = reg_Ridge.predict(df_feature_test)


Calculating accuracy measures

In [ ]:
print("r2_score:_ Ridge",r2_score(df_y_test, y_pred_list_Ridge))
print("explained_variance_score:Ridge ",explained_variance_score(df_y_test, y_pred_list_Ridge))
print("mean_absolute_error:Ridge ",mean_absolute_error(df_y_test, y_pred_list_Ridge))
print("mean_squared_error:Ridge ",mean_squared_error(df_y_test, y_pred_list_Ridge))
print("median_absolute_error:Ridge ",median_absolute_error(df_y_test, y_pred_list_Ridge))
# print("mean_squared_log_error:Ridge ",mean_squared_log_error(df_y_test, y_pred_list))
